In [31]:
## Import required dependencies
import os
import logging 
import requests 
import psycopg2
import pandas as pd 
from dotenv import load_dotenv
from requests.exceptions import RequestException
from sqlalchemy import create_engine

## Load environment variables
load_dotenv()

API_KEY         =   os.getenv("API_KEY")
API_HOST        =   os.getenv("API_HOST")
DB_NAME         =   os.getenv("DB_NAME")
DB_USER     =   os.getenv("DB_USER")
DB_PASS         =   os.getenv("DB_PASS")
DB_HOST         =   os.getenv("DB_HOST")
DB_PORT         =   int(os.getenv("DB_PORT"))


## Set up logger
logging.basicConfig(level=logging.DEBUG, format = '%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

## Set up file and console handlers
file_handler = logging.FileHandler('standings.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(file_handler)

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(console_handler)


##Define API endpoints, headers and query parameters
url = "https://api-football-v1.p.rapidapi.com/v3/standings"
querystring = {"season":2020, "league":39}
headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": API_HOST
}


try:
    api_response = api_response = requests.get(url, headers=headers, params=querystring, timeout=20)
    api_response.raise_for_status() 


except HTTPError as http_err:
    logger.error(f'HTTP error occurred: {http_err}')


except Timeout:
    logger.error('Request timed out after 20 seconds')


except RequestException as request_err:
    logger.error(f'Request error occurred: {request_err}')




2024-06-26 10:27:42,364 - DEBUG - Starting new HTTPS connection (1): api-football-v1.p.rapidapi.com:443
2024-06-26 10:27:42,364 - DEBUG - Starting new HTTPS connection (1): api-football-v1.p.rapidapi.com:443
2024-06-26 10:27:42,364 - DEBUG - Starting new HTTPS connection (1): api-football-v1.p.rapidapi.com:443
2024-06-26 10:27:42,364 - DEBUG - Starting new HTTPS connection (1): api-football-v1.p.rapidapi.com:443
2024-06-26 10:27:42,364 - DEBUG - Starting new HTTPS connection (1): api-football-v1.p.rapidapi.com:443
2024-06-26 10:27:42,364 - DEBUG - Starting new HTTPS connection (1): api-football-v1.p.rapidapi.com:443
2024-06-26 10:27:42,364 - DEBUG - Starting new HTTPS connection (1): api-football-v1.p.rapidapi.com:443
2024-06-26 10:27:42,364 - DEBUG - Starting new HTTPS connection (1): api-football-v1.p.rapidapi.com:443
2024-06-26 10:27:42,593 - DEBUG - https://api-football-v1.p.rapidapi.com:443 "GET /v3/standings?season=2020&league=39 HTTP/1.1" 200 None
2024-06-26 10:27:42,593 - DEBUG

In [18]:
DB_USER

'qizwgtmh'

In [3]:
standings = api_response.json()['response']

In [4]:
standings_data = standings[0]['league']['standings'][0]
standings_data

[{'rank': 1,
  'team': {'id': 50,
   'name': 'Manchester City',
   'logo': 'https://media.api-sports.io/football/teams/50.png'},
  'points': 86,
  'goalsDiff': 51,
  'group': 'Premier League',
  'form': 'WLWLW',
  'status': 'same',
  'description': 'Promotion - Champions League (Group Stage)',
  'all': {'played': 38,
   'win': 27,
   'draw': 5,
   'lose': 6,
   'goals': {'for': 83, 'against': 32}},
  'home': {'played': 19,
   'win': 13,
   'draw': 2,
   'lose': 4,
   'goals': {'for': 43, 'against': 17}},
  'away': {'played': 19,
   'win': 14,
   'draw': 3,
   'lose': 2,
   'goals': {'for': 40, 'against': 15}},
  'update': '2021-05-23T00:00:00+00:00'},
 {'rank': 2,
  'team': {'id': 33,
   'name': 'Manchester United',
   'logo': 'https://media.api-sports.io/football/teams/33.png'},
  'points': 74,
  'goalsDiff': 29,
  'group': 'Premier League',
  'form': 'WDLLW',
  'status': 'same',
  'description': 'Promotion - Champions League (Group Stage)',
  'all': {'played': 38,
   'win': 21,
   'd

In [5]:
data = []

for team_details in standings_data:
    rank            =   team_details['rank']
    name            =   team_details['team']['name']
    played          =   team_details['all']['played']
    win             =   team_details['all']['win']
    draw            =   team_details['all']['draw']
    lose            =   team_details['all']['lose']
    goals_for       =   team_details['all']['goals']['for']
    goals_against   =   team_details['all']['goals']['against']
    goals_diff      =   team_details['goalsDiff']
    points          =   team_details['points']
    
    data.append([rank,name,played,win,draw,lose,goals_for,goals_against,goals_diff,points])
    
data

[[1, 'Manchester City', 38, 27, 5, 6, 83, 32, 51, 86],
 [2, 'Manchester United', 38, 21, 11, 6, 73, 44, 29, 74],
 [3, 'Liverpool', 38, 20, 9, 9, 68, 42, 26, 69],
 [4, 'Chelsea', 38, 19, 10, 9, 58, 36, 22, 67],
 [5, 'Leicester', 38, 20, 6, 12, 68, 50, 18, 66],
 [6, 'West Ham', 38, 19, 8, 11, 62, 47, 15, 65],
 [7, 'Tottenham', 38, 18, 8, 12, 68, 45, 23, 62],
 [8, 'Arsenal', 38, 18, 7, 13, 55, 39, 16, 61],
 [9, 'Leeds', 38, 18, 5, 15, 62, 54, 8, 59],
 [10, 'Everton', 38, 17, 8, 13, 47, 48, -1, 59],
 [11, 'Aston Villa', 38, 16, 7, 15, 55, 46, 9, 55],
 [12, 'Newcastle', 38, 12, 9, 17, 46, 62, -16, 45],
 [13, 'Wolves', 38, 12, 9, 17, 36, 52, -16, 45],
 [14, 'Crystal Palace', 38, 12, 8, 18, 41, 66, -25, 44],
 [15, 'Southampton', 38, 12, 7, 19, 47, 68, -21, 43],
 [16, 'Brighton', 38, 9, 14, 15, 40, 46, -6, 41],
 [17, 'Burnley', 38, 10, 9, 19, 33, 55, -22, 39],
 [18, 'Fulham', 38, 5, 13, 20, 27, 53, -26, 28],
 [19, 'West Brom', 38, 5, 11, 22, 35, 76, -41, 26],
 [20, 'Sheffield Utd', 38, 7, 2, 2

In [6]:
df = pd.DataFrame(data, columns=['rank','name','played','win','draw','lose','goals_for','goals_against','goals_diff','points'])
df


,rank,name,played,win,draw,lose,goals_for,goals_against,goals_diff,points
0,1,Manchester City,38,27,5,6,83,32,51,86
1,2,Manchester United,38,21,11,6,73,44,29,74
2,3,Liverpool,38,20,9,9,68,42,26,69
3,4,Chelsea,38,19,10,9,58,36,22,67
4,5,Leicester,38,20,6,12,68,50,18,66
5,6,West Ham,38,19,8,11,62,47,15,65
6,7,Tottenham,38,18,8,12,68,45,23,62
7,8,Arsenal,38,18,7,13,55,39,16,61
8,9,Leeds,38,18,5,15,62,54,8,59
9,10,Everton,38,17,8,13,47,48,-1,59


In [32]:
# Create the database connection string
connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Name of the table to write to
table_name = 'premier_league_standings'

try:
    # Write the DataFrame to the PostgreSQL table
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    logger.info(f'DataFrame written to {table_name} table in {DB_NAME} database')

except Exception as e:
    logger.error(f'Error writing DataFrame to {table_name} table in {DB_NAME} database: {e}')


2024-06-26 10:27:51,516 - ERROR - Error writing DataFrame to premier_league_standings table in qizwgtmh database: Unable to find a usable engine; tried using: 'sqlalchemy'.
A suitable version of sqlalchemy is required for sql I/O support.
Trying to import the above resulted in these errors:
 - Pandas requires version '1.4.16' or newer of 'sqlalchemy' (version '1.3.18' currently installed).
2024-06-26 10:27:51,516 - ERROR - Error writing DataFrame to premier_league_standings table in qizwgtmh database: Unable to find a usable engine; tried using: 'sqlalchemy'.
A suitable version of sqlalchemy is required for sql I/O support.
Trying to import the above resulted in these errors:
 - Pandas requires version '1.4.16' or newer of 'sqlalchemy' (version '1.3.18' currently installed).
2024-06-26 10:27:51,516 - ERROR - Error writing DataFrame to premier_league_standings table in qizwgtmh database: Unable to find a usable engine; tried using: 'sqlalchemy'.
A suitable version of sqlalchemy is requi